In [13]:
spark.sparkContext.stop

In [3]:
import org.apache.spark.sql.{DataFrame, SparkSession}
import org.apache.spark._
import org.apache.spark.rdd.RDD
import spark.implicits._


val SPARK_NAMESPACE="default"
val SA="spark-driver"
val K8S_CACERT="/var/run/secrets/kubernetes.io/serviceaccount/ca.crt"
val K8S_TOKEN="/var/run/secrets/kubernetes.io/serviceaccount/token"
val DOCKER_IMAGE="hishailesh77/spark2.4.6-deb:latest"
val SPARK_DRIVER_HOST= "jupyter-lab-host.jupyter-headless.default.svc.cluster.local"
val SPARK_DRIVER_PORT="20020"

import org.apache.spark.sql.{DataFrame, SparkSession}
import org.apache.spark._
import org.apache.spark.rdd.RDD
import spark.implicits._
SPARK_NAMESPACE: String = default
SA: String = spark-driver
K8S_CACERT: String = /var/run/secrets/kubernetes.io/serviceaccount/ca.crt
K8S_TOKEN: String = /var/run/secrets/kubernetes.io/serviceaccount/token
DOCKER_IMAGE: String = hishailesh77/spark2.4.6-deb:latest
SPARK_DRIVER_HOST: String = jupyter-lab-host.jupyter-headless.default.svc.cluster.local
SPARK_DRIVER_PORT: String = 20020


In [5]:
val spark = SparkSession
.builder()
.appName("Spark-Spylon")
.master("k8s://https://kubernetes.default:443")
.config("spark.kubernetes.authenticate.driver.serviceAccountName",SA)
.config("spark.kubernetes.namespace",SPARK_NAMESPACE)
.config("spark.kubernetes.authenticate.subdmission.caCertFile",K8S_CACERT)
.config("spark.kubernetes.authenticate.submission.oauthTokenFile",K8S_TOKEN)
.config("spark.kubernetes.container.image", DOCKER_IMAGE)
.config("spark.kubernetes.container.image.pullPolicy","Always")
.config("spark.driver.port",SPARK_DRIVER_PORT)
.config("spark.driver.host",SPARK_DRIVER_HOST)
.config("spark.executor.instances", "25")
.config("spark.driver.memory","16g")
.config("spark.executor.memory","8g")
.config("spark.driver.cores","8")
.config("spark.executor.cores","2")
.config("spark.hadoop.fs.s3a.access.key", "")
.config("spark.hadoop.fs.s3a.secret.key", "")
.config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
.config("spark.hadoop.fs.s3a.multiobjectdelete.enable","false")
.config("spark.hadoop.fs.s3a.fast.upload","true")
.getOrCreate()

spark: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@63ab6dc1


In [6]:
val malware_list_rdd = spark.sparkContext.wholeTextFiles("s3a://ml-workflow-data/security/Malware_Dataset/Malware_Classification_DataSet/*.asm", 500)

malware_list_rdd: org.apache.spark.rdd.RDD[(String, String)] = s3a://ml-workflow-data/security/Malware_Dataset/Malware_Classification_DataSet/*.asm MapPartitionsRDD[1] at wholeTextFiles at <console>:34


In [7]:
val malware_df = spark.createDataFrame(malware_list_rdd).toDF("File", "Text").
withColumn("file_name",split(col("File"),"/").
getItem(6)).withColumn("file_name", regexp_replace($"file_name", ".asm" , "") )

malware_df: org.apache.spark.sql.DataFrame = [File: string, Text: string ... 1 more field]


In [8]:
malware_df.rdd.getNumPartitions

res2: Int = 238


In [9]:
malware_df.show()

+--------------------+--------------------+--------------------+
|                File|                Text|           file_name|
+--------------------+--------------------+--------------------+
|s3a://ml-workflow...|HEADER:10000000		...|01kcPWA9K2BOxQeS5Rju|
|s3a://ml-workflow...|HEADER:4DF30000		...|04EjIdbPV5e1XroFOpiN|
|s3a://ml-workflow...|HEADER:10000000		...|05EeG39MTRrI6VY21DPd|
|s3a://ml-workflow...|HEADER:7DF00000		...|05rJTUWYAKNegBk2wE8X|
|s3a://ml-workflow...|HEADER:10000000		...|0AnoOZDNbPXIr2MRBSCJ|
|s3a://ml-workflow...|HEADER:675A0000		...|0AwWs42SUQ19mI7eDcTC|
|s3a://ml-workflow...|HEADER:10000000		...|0DNVFKwYlcjO7bTfJ5p1|
|s3a://ml-workflow...|HEADER:10000000		...|0DqUX5rkg3IbMY6BLGCE|
|s3a://ml-workflow...|HEADER:00400000		...|0G4hwobLuAzvl1PWYfmd|
|s3a://ml-workflow...|HEADER:00400000		...|0H63jydvIahOVqgx5Kfo|
|s3a://ml-workflow...|HEADER:67AE0000		...|0Hrfce4X5YGESJPjl9uL|
|s3a://ml-workflow...|HEADER:6D6D0000		...|0I4ZVvngsAatm8fzD3pk|
|s3a://ml-workflow...|HEA

In [10]:
val fileName_df= spark.read.format("csv").option("inferSchema", "true").
    option("header", "true").
    load("s3a://ml-workflow-data/security/Malware_Dataset/Malware_Classification_DataSet/Training_Label_Classification_SmallSet.csv")

fileName_df: org.apache.spark.sql.DataFrame = [file_name: string, label: int]


In [11]:
val mergedDf = malware_df.join(fileName_df, Seq("file_name"), "inner")

mergedDf: org.apache.spark.sql.DataFrame = [file_name: string, File: string ... 2 more fields]


In [12]:
mergedDf.write.mode("overwrite").format("parquet").option("compression", "snappy").mode("overwrite").save("s3a://ml-workflow-data/security/Malware_Dataset/Output/malware_tmp_parquet_1")